# Q-LEARNING FOR THE MAZE GAME
    Author: Furkan Cantürk
    Date: 01.09.2020

In [1]:
import numpy as np
from random import randint
import random

In [4]:
def select_action(current_state, p):
    global current_pos, epsilon
    possible_actions = []
    if np.random.uniform() <= epsilon:
        if current_pos[0] != 0: possible_actions.append(0)
        if current_pos[0] != n-1: possible_actions.append(1)
        if current_pos[1] != 0: possible_actions.append(2)
        if current_pos[1] != n-1: possible_actions.append(3)
        
        intended_action = possible_actions[randint(0, len(possible_actions) - 1)]
        
        if np.random.uniform() > p: # p is given as 0 for deterministic case, otherwise 0.5
            return intended_action
        else:
            possible_actions.remove(intended_action)
            return possible_actions[randint(0, len(possible_actions) - 1)]

    else:
        if current_pos[0] != 0: possible_actions.append(Q[current_state,0])
        else: possible_actions.append(-999)
            
        if current_pos[0] != n-1: possible_actions.append(Q[current_state,1])
        else: possible_actions.append(-999)
            
        if current_pos[1] != 0: possible_actions.append(Q[current_state,2])
        else: possible_actions.append(-999)
            
        if current_pos[1] != n-1: possible_actions.append(Q[current_state,3])
        else: possible_actions.append(-999)
        # choose action randomly if there are multiple max Q values  
        intended_action = random.choice([i for i, a in enumerate(possible_actions) if a == max(possible_actions)])
        
        if np.random.uniform() > p: # p is given as 0 for deterministic case, otherwise 0.5
            return intended_action
        else:
            possible_actions = [i for i, a in enumerate(possible_actions) if a != -999]
            possible_actions.remove(intended_action)
            return random.choice(possible_actions)

In [2]:
def off_policy_episode(): # probability of not intended actions is 0 
    global current_pos, epsilon
    current_pos = [n-1,0] # start
    current_state = states[(current_pos[0], current_pos[1])]
    
    while current_state != terminal_state:
        action = select_action(current_state, 0) # deterministic
        #take the  action
        if action == 0: current_pos[0] -= 1
        elif action == 1: current_pos[0] += 1
        elif action == 2: current_pos[1] -= 1
        elif action == 3: current_pos[1] += 1
        
        r = reward[current_pos[0],current_pos[1]]
        new_state = states[(current_pos[0], current_pos[1])]
        
        Q[current_state, action] += l_rate*( r + gamma * np.max(Q[new_state]) - Q[current_state,action])
        current_state = new_state
        epsilon -= 0.001

In [3]:
def on_policy_episode(): # probability of intended action is 0.5 
    global current_pos, epsilon
    current_pos = [n-1,0] # start
    current_state = states[(current_pos[0], current_pos[1])]
    action = select_action(current_state, 0.5) # undeterministic
    
    while current_state != terminal_state:
        #take the action
        if action == 0: current_pos[0] -= 1
        elif action == 1: current_pos[0] += 1
        elif action == 2: current_pos[1] -= 1
        elif action == 3: current_pos[1] += 1
        r = reward[current_pos[0],current_pos[1]]
        new_state = states[(current_pos[0], current_pos[1])]
        new_action = select_action(new_state, 0.5) # undeterministic
        
        Q[current_state, action] += l_rate*( r + gamma * Q[new_state, new_action] - Q[current_state,action])
        
        current_state = new_state
        action = new_action
        epsilon -= 0.001

In [5]:
def get_next_state(state):
    act = np.argmax(Q[state])
    if act == 0: return state - 8
    elif act == 1: return state + 8
    elif act == 2: return state - 1
    elif act == 3: return state + 1

In [6]:
def path():
    path = []
    next_s = states[(start[0],start[1])]
    while next_s != states[(terminal[0], terminal[1])]:
        path.append(next_s)
        next_s = get_next_state(next_s)
    path.append(next_s)
    return path

In [7]:
n = 8
tx = 6
ty = 2
terminal = [ty,tx]
start = [n-1,0]
maze = np.zeros([n,n])

l_rate = 0.1
gamma = 0.9

directions = {"up": 0, "down" : 1, "left" : 2, "right" : 3}
states = {}
k = 0
for i in range(n):
    for j in range(n):
        states[(i,j)] = k
        maze[i,j] = k
        k+=1
terminal_state = states[(terminal[0], terminal[1])]

### OFF-POLICY - DETERMINISTIC ACTIONS

In [18]:
random.seed(42)
np.random.seed(42)
reward = np.zeros((n,n))
reward[ty,tx] = 100
Q = np.zeros((n**2,4))
epsilon = 0.7
for i in range(1000):
    off_policy_episode()

In [19]:
p = path()
result_maze = maze.copy()
for i in range(n):
    for j in range(n):
        if maze[i, j] not in p:
            result_maze[i,j] = None
print("States through the off-policy path:")            
result_maze

States through the off-policy path:


array([[nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, 18., 19., 20., 21., 22., nan],
       [nan, nan, 26., nan, nan, nan, nan, nan],
       [nan, nan, 34., nan, nan, nan, nan, nan],
       [40., 41., 42., nan, nan, nan, nan, nan],
       [48., nan, nan, nan, nan, nan, nan, nan],
       [56., nan, nan, nan, nan, nan, nan, nan]])

In [20]:
best_acts = np.zeros((n,n))
for i in range(n):
    for j in range(n):
        if(sum(Q[states[(i,j)]]!=0)):
            best_acts[i,j] = np.argmax(Q[states[(i,j)]])
        else:
            best_acts[i,j]= None

In [21]:
best_acts  # directions = {"up": 0, "down" : 1, "left" : 2, "right" : 3}

array([[nan, nan,  1., nan, nan, nan, nan, nan],
       [nan, nan,  1., nan, nan, nan, nan, nan],
       [nan, nan,  3.,  3.,  3.,  3., nan,  2.],
       [nan, nan,  0.,  0., nan,  0.,  0.,  2.],
       [nan, nan,  0., nan,  3.,  3.,  0., nan],
       [ 3.,  3.,  0., nan,  0., nan, nan, nan],
       [ 0.,  0.,  0.,  3.,  0.,  2., nan, nan],
       [ 0., nan, nan,  0., nan, nan, nan, nan]])

In [22]:
np.around(Q) # off-policy

array([[  0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,  73.],
       [  0.,   0.,   0.,  81.],
       [  0.,   0.,   0.,  90.],
       [  0.,   0.,   0., 100.],
       [  0.,   0.,   0.,   0.],
       [  0.,   0.,  10.,   0.],
       [  0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.],
       [ 66.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.],
       [  4.,   0.,   0.,   0.],
       [ 6

### ON-POLICY - UNDETERMINISTIC ACTIONS

In [23]:
random.seed(42)
np.random.seed(42)
reward = np.zeros((n,n))
reward[ty,tx] = 100
Q = np.zeros((n**2,4))
epsilon = 0.7
for i in range(1000):
    on_policy_episode()

In [24]:
p = path()
result_maze = maze.copy()
for i in range(n):
    for j in range(n):
        if maze[i, j] not in p:
            result_maze[i,j] = None
print("States through the on-policy path:")            
result_maze

States through the on-policy path:


array([[nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, 21., 22., nan],
       [nan, nan, nan, nan, nan, 29., nan, nan],
       [nan, nan, nan, 35., 36., 37., nan, nan],
       [nan, nan, 42., 43., nan, nan, nan, nan],
       [48., 49., 50., nan, nan, nan, nan, nan],
       [56., nan, nan, nan, nan, nan, nan, nan]])

In [25]:
best_acts = np.zeros((n,n))
for i in range(n):
    for j in range(n):
        if(sum(Q[states[(i,j)]]!=0)):
            best_acts[i,j] = np.argmax(Q[states[(i,j)]])
        else:
            best_acts[i,j]= None

In [26]:
best_acts  # directions = {"up": 0, "down" : 1, "left" : 2, "right" : 3}

array([[ 1.,  3.,  3.,  3.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  3.,  3.,  3.,  3.,  3., nan,  2.],
       [ 3.,  3.,  3.,  3.,  3.,  0.,  0.,  0.],
       [ 3.,  3.,  3.,  3.,  3.,  0.,  0.,  0.],
       [ 3.,  3.,  3.,  0.,  0.,  0.,  0.,  0.],
       [ 3.,  3.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  3.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [27]:
np.around(Q) # on-policy

array([[  0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.],
       [  0.,   3.,   0.,   5.],
       [  0.,  12.,   1.,  16.],
       [  0.,  29.,   8.,  11.],
       [  0.,  39.,   8.,   8.],
       [  0.,  46.,   4.,   7.],
       [  0.,  22.,  11.,   0.],
       [  0.,   1.,   0.,   1.],
       [  0.,   6.,   0.,   2.],
       [  1.,  15.,   2.,  11.],
       [  8.,  27.,   5.,  25.],
       [ 11.,  43.,  13.,  39.],
       [ 21.,  69.,  31.,  39.],
       [  7., 100.,  28.,  26.],
       [  6.,  57.,  44.,   0.],
       [  0.,   4.,   0.,   3.],
       [  2.,   7.,   1.,  16.],
       [  4.,  18.,   5.,  32.],
       [ 13.,  31.,  20.,  46.],
       [ 27.,  39.,  31.,  72.],
       [ 48.,  49.,  48., 100.],
       [  0.,   0.,   0.,   0.],
       [ 35.,  40., 100.,   0.],
       [  1.,   5.,   0.,  12.],
       [  5.,   9.,   4.,  15.],
       [ 18.,  15.,   9.,  27.],
       [ 23.,  22.,  17.,  39.],
       [ 43.,  29.,  28.,  58.],
       [ 77.,  39.,  38.,  67.],
       [10